In [1]:
%pip install quantstats
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [67]:
import quantstats as qs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.linear_model import LinearRegression

In [68]:
# Change day and round here
path1 = f"./data/day_0/prices_round_1_day_-2.csv"
path2 = f"./data/day_0/prices_round_1_day_-1.csv"
path3 = f"./data/day_0/prices_round_1_day_0.csv"

In [69]:
# Get data
data1 = pd.read_csv(path1, delimiter=";", parse_dates=True)
data2 = pd.read_csv(path2, delimiter=";", parse_dates=True)
data3 = pd.read_csv(path3, delimiter=";", parse_dates=True)

In [70]:
parsed_data_1 = data1[['day', 'timestamp', 'product', 'mid_price']].copy()
parsed_data_2 = data2[['day', 'timestamp', 'product', 'mid_price']].copy()
parsed_data_3 = data3[['day', 'timestamp', 'product', 'mid_price']].copy()

In [71]:
parsed_data = pd.concat([parsed_data_2, parsed_data_3], ignore_index=True)
parsed_data

,day,timestamp,product,mid_price
0,-1,0,AMETHYSTS,10003.0
1,-1,0,STARFRUIT,5046.0
2,-1,100,STARFRUIT,5043.0
3,-1,100,AMETHYSTS,10000.0
4,-1,200,AMETHYSTS,10000.0
...,...,...,...,...
39995,0,999700,STARFRUIT,5053.5
39996,0,999800,STARFRUIT,5049.5
39997,0,999800,AMETHYSTS,10000.0
39998,0,999900,STARFRUIT,5051.0


In [72]:
def calculate_price_index(p1, p2, p3, type):
    l = [p1, p2, p3]
    
    for i in range(len(l)):
        if not isinstance(l[i], int):
            del l[i]

    if type == 'bid':
        return l.index(max(l))
    else:
        return l.index(min(l))

In [73]:
def add_lags(parsed_data):
    parsed_data.drop(parsed_data[parsed_data["product"] == "AMETHYSTS"].index, inplace=True)
    lags = 4
    for i in range(1, lags + 1):
        parsed_data[f'lag{i}'] = parsed_data['mid_price'].shift(i)

    parsed_data.dropna(inplace=True)
    return parsed_data

parsed_data = add_lags(parsed_data)
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4
9,-1,400,STARFRUIT,5042.5,5043.0,5044.0,5043.0,5046.0
10,-1,500,STARFRUIT,5042.5,5042.5,5043.0,5044.0,5043.0
13,-1,600,STARFRUIT,5044.0,5042.5,5042.5,5043.0,5044.0
15,-1,700,STARFRUIT,5044.0,5044.0,5042.5,5042.5,5043.0
17,-1,800,STARFRUIT,5042.5,5044.0,5044.0,5042.5,5042.5
...,...,...,...,...,...,...,...,...
39990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0
39992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5
39995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0
39996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5


In [74]:
x = parsed_data[['lag1', 'lag2', 'lag3', 'lag4']]
# x = parsed_data[['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10']]
x

,lag1,lag2,lag3,lag4
9,5043.0,5044.0,5043.0,5046.0
10,5042.5,5043.0,5044.0,5043.0
13,5042.5,5042.5,5043.0,5044.0
15,5044.0,5042.5,5042.5,5043.0
17,5044.0,5044.0,5042.5,5042.5
...,...,...,...,...
39990,5051.5,5052.0,5051.5,5052.0
39992,5052.5,5051.5,5052.0,5051.5
39995,5051.0,5052.5,5051.5,5052.0
39996,5053.5,5051.0,5052.5,5051.5


In [75]:
parsed_data_test = add_lags(parsed_data_1)
parsed_data_test

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4
9,-2,400,STARFRUIT,4998.5,4999.5,5000.0,5002.0,5000.0
11,-2,500,STARFRUIT,5002.0,4998.5,4999.5,5000.0,5002.0
12,-2,600,STARFRUIT,4999.5,5002.0,4998.5,4999.5,5000.0
15,-2,700,STARFRUIT,5000.0,4999.5,5002.0,4998.5,4999.5
17,-2,800,STARFRUIT,5002.5,5000.0,4999.5,5002.0,4998.5
...,...,...,...,...,...,...,...,...
19990,-2,999500,STARFRUIT,5044.0,5044.0,5043.0,5043.5,5044.5
19993,-2,999600,STARFRUIT,5043.5,5044.0,5044.0,5043.0,5043.5
19994,-2,999700,STARFRUIT,5043.5,5043.5,5044.0,5044.0,5043.0
19996,-2,999800,STARFRUIT,5043.0,5043.5,5043.5,5044.0,5044.0


In [81]:
x_test = parsed_data_test[['lag1', 'lag2', 'lag3', 'lag4']]
# x_test = parsed_data_test[['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'lag6', 'lag7', 'lag8', 'lag9', 'lag10']]
x_test

,lag1,lag2,lag3,lag4
9,4999.5,5000.0,5002.0,5000.0
11,4998.5,4999.5,5000.0,5002.0
12,5002.0,4998.5,4999.5,5000.0
15,4999.5,5002.0,4998.5,4999.5
17,5000.0,4999.5,5002.0,4998.5
...,...,...,...,...
19990,5044.0,5043.0,5043.5,5044.5
19993,5044.0,5044.0,5043.0,5043.5
19994,5043.5,5044.0,5044.0,5043.0
19996,5043.5,5043.5,5044.0,5044.0


In [77]:
y = parsed_data['mid_price']
y

9        5042.5
10       5042.5
13       5044.0
15       5044.0
17       5042.5
          ...  
39990    5052.5
39992    5051.0
39995    5053.5
39996    5049.5
39998    5051.0
Name: mid_price, Length: 19996, dtype: float64

In [78]:
y_test = parsed_data_1['mid_price']
y_test

9        4998.5
11       5002.0
12       4999.5
15       5000.0
17       5002.5
          ...  
19990    5044.0
19993    5043.5
19994    5043.5
19996    5043.0
19998    5045.5
Name: mid_price, Length: 9996, dtype: float64

In [79]:
model = LinearRegression()
model.fit(x, y)
pred = model.predict(x)
parsed_data["pred_price"] = pred
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4,pred_price
9,-1,400,STARFRUIT,5042.5,5043.0,5044.0,5043.0,5046.0,5043.842033
10,-1,500,STARFRUIT,5042.5,5042.5,5043.0,5044.0,5043.0,5043.067459
13,-1,600,STARFRUIT,5044.0,5042.5,5042.5,5043.0,5044.0,5042.910324
15,-1,700,STARFRUIT,5044.0,5044.0,5042.5,5042.5,5043.0,5043.134914
17,-1,800,STARFRUIT,5042.5,5044.0,5044.0,5042.5,5042.5,5043.436610
...,...,...,...,...,...,...,...,...,...
39990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0,5051.730655
39992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5,5051.954463
39995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0,5051.690827
39996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5,5052.270074


In [82]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Accuracy:", model.score(x_test, y_test))

Coefficients: [0.34199013 0.26233351 0.20957588 0.18360716]
Intercept: 12.602735193926492
Accuracy: 0.9981111930940674
